In [ ]:
import torch
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

from nemo.global_planner import AStarGradPlanner
from nemo.nemo import Nemo
from nemo.util import wrap_angle_torch, path_metrics
from nemo.plotting import plot_surface, plot_path_3d
from nemo.planning import path_optimization

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

%load_ext autoreload
%autoreload 2

## Load the heightnet

In [ ]:
nemo = Nemo()
nemo.load_weights('../models/AirSimMountains/AirSimMountains_encs.pth', '../models/AirSimMountains/AirSimMountains_mlp.pth')

In [ ]:
N = 512
bounds = np.array([-0.75, 0.45, -0.6, 0.6])
xs = torch.linspace(bounds[0], bounds[1], N, device=device)
ys = torch.linspace(bounds[2], bounds[3], N, device=device)
XY_grid = torch.meshgrid(xs, ys, indexing='xy')
XY_grid = torch.stack(XY_grid, dim=-1)
positions = XY_grid.reshape(-1, 2)

heights = nemo.get_heights(positions)

In [ ]:
z_grid = heights.reshape(N, N).detach().cpu().numpy()
x_grid = XY_grid[:,:,0].detach().cpu().numpy()
y_grid = XY_grid[:,:,1].detach().cpu().numpy()

fig = plot_surface(x_grid, y_grid, z_grid, no_axes=False, showscale=False)
fig.update_layout(width=1600, height=900)
fig.show()

## Coordinate transforms

In [ ]:
import json

dataparser_transforms = json.load(open('../models/AirSimMountains/dataparser_transforms.json'))
transform = np.array(dataparser_transforms['transform'])
scale = dataparser_transforms['scale']

In [ ]:
# Convert AirSim coordinates to Nemo coordinates
airsim_start = np.array([177., -247., -33.])  # AirSim global
airsim_end = airsim_start + np.array([-192., -328., -68.])  # AirSim global
center = np.array([99., -449., -57.])  # AirSim
temp = airsim_start - center
data_start = np.array([temp[1], temp[0], -temp[2]])
temp = airsim_end - center
data_end = np.array([temp[1], temp[0], -temp[2]])

ns_start = scale * (data_start + transform[0:3,3])
ns_end = scale * (data_end + transform[0:3,3])
print(ns_start, ns_end)

In [ ]:
np.linalg.norm(airsim_end - airsim_start)

## Initialization

In [ ]:
# Initialize the planner with scaled heightmap
astar_heights = 1e2 * (z_grid + 1.0).reshape(N, N)
gp = AStarGradPlanner(astar_heights, bounds)

# Start and end positions for path
start = tuple(ns_start[:2])
end = tuple(ns_end[:2])

# Compute path
path_xy = gp.spatial_plan(start, end)
path_xy_torch = torch.tensor(path_xy, device=device)
# Get heights along path
path_zs = nemo.get_heights(path_xy_torch)  

# Save path as torch tensor
astar_path = torch.cat((path_xy_torch, path_zs), dim=1)

init_xy = path_xy_torch

In [ ]:
N_path = 150

# init_xy is (N, 2) tensor, straight line between start and end
init_xy = torch.stack((torch.linspace(ns_start[0], ns_end[0], N_path, device=device), 
                       torch.linspace(ns_start[1], ns_end[1], N_path, device=device))).T

theta_0 = torch.pi

## Path optimization

In [ ]:
from nemo.dynamics import diff_flatness, compute_slopes_sample

In [ ]:
path_start = init_xy[0]
path_end = init_xy[-1]
path_opt = init_xy[1:-1].clone().detach().requires_grad_(True)  # portion of the path to optimize
path = torch.cat((path_start[None], path_opt, path_end[None]), dim=0)  # full path

In [ ]:
xy = path
dt = 1.0

x = xy[:, 0]
y = xy[:, 1]
epsilon = torch.tensor(1e-3, device=device, requires_grad=True)
xdot = torch.hstack((epsilon, torch.diff(x) / dt))
ydot = torch.hstack((epsilon, torch.diff(y) / dt))
xddot = torch.hstack((epsilon, torch.diff(xdot) / dt))
yddot = torch.hstack((epsilon, torch.diff(ydot) / dt))
v = torch.sqrt(xdot**2 + ydot**2)
theta = torch.arctan2(ydot, xdot)

In [ ]:
l = scale * 4.0  # vehicle length 
dl = l/2 * torch.stack((torch.cos(theta), torch.sin(theta))).T 
z_front = nemo.get_heights(xy + dl).float()
z_back = nemo.get_heights(xy - dl).float()
phi = torch.arctan2(z_front - z_back, torch.tensor(l))

In [ ]:
#g_eff = 9.81 * torch.sin(phi)
#g_eff = torch.tensor(9.81, device=device) * torch.sin(phi)
g_eff = torch.sin(phi)

In [ ]:
u = torch.zeros(len(x), 2)
for i in range(len(x)):
    J = torch.tensor([[torch.cos(theta[i]), -v[i] * torch.sin(theta[i])],
                      [torch.sin(theta[i]), v[i] * torch.cos(theta[i])]], device=device, requires_grad=True)
    b = torch.tensor([[xddot[i] + g_eff[i] * torch.cos(theta[i])],
                      [yddot[i] + g_eff[i] * torch.sin(theta[i])]], device=device, requires_grad=True)
    u[i] = torch.linalg.solve(J, b).flatten()

In [ ]:
b

In [ ]:
# Formulate cost and compute gradients
c = torch.sum(b)
c.backward()
path_opt_grad = path_opt.grad

# Debugging: Check gradients
print("path_opt.grad:", path_opt_grad)

In [ ]:
# _, grad = nemo.get_heights_with_grad(xy.clone().requires_grad_(True))
# psi = torch.atan2(grad[:,1], grad[:,0])
# alpha = torch.atan(grad.norm(dim=1))

# phi = alpha * torch.cos(theta - psi)

l = scale * 4.0  # vehicle length 
phi = compute_slopes_sample(xy, theta, nemo, l)

g_eff = 9.81 * torch.sin(phi)

u = torch.zeros(len(x), 2)
for i in range(len(x)):
    J = torch.tensor([[torch.cos(theta[i]), -v[i] * torch.sin(theta[i])],
                      [torch.sin(theta[i]), v[i] * torch.cos(theta[i])]], device=device, requires_grad=True)
    b = torch.tensor([[xddot[i] + g_eff[i] * torch.cos(theta[i])],
                      [yddot[i] + g_eff[i] * torch.sin(theta[i])]], device=device, requires_grad=True)
    u[i] = torch.linalg.solve(J, b).flatten()

In [ ]:
# Dubin's based cost
def cost(path, dt=1.0):
    u = diff_flatness(path, nemo, dt)
    return torch.sum(u[:,0]**2 + u[:,1]**2)

In [ ]:
c = cost(path)

In [ ]:
c.backward()

In [ ]:
path_opt.grad

In [ ]:
# Optimize path
opt = torch.optim.Adam([path_opt], lr=1e-3)

In [ ]:
iterations = 1000

for it in range(iterations):
    opt.zero_grad()
    path = torch.cat((path_start[None], path_opt, path_end[None]), dim=0)
    c = cost(path)
    c.backward()
    opt.step()
    if it % 50 == 0:
        print(f'it: {it},  Cost: {c.item()}')

print(f'Finished optimization - final cost: {c.item()}')

# Compute final heights
path_zs = nemo.get_heights(path)

# Full 3D path
path_3d = torch.cat((path, path_zs), dim=1)

In [ ]:
fig = plot_surface(x_grid, y_grid, z_grid, no_axes=False)
fig = plot_path_3d(fig=fig, x=path_3d[:,0].detach().cpu().numpy(), 
                        y=path_3d[:,1].detach().cpu().numpy(), 
                        z=path_3d[:,2].detach().cpu().numpy(),
                        markers=False)
fig.show()

In [ ]:
# Convert Nemo coordinates to AirSim coordinates (local)
temp = ns_start / scale - transform[0:3,3]
temp[[0,1]] = temp[[1,0]]
temp[2] = -temp[2]
temp = temp + center

In [ ]:
airsim_path_3d = path_3d.detach().cpu().numpy()
airsim_path_3d = airsim_path_3d / scale - transform[0:3,3]
airsim_path_3d[:,[0,1]] = airsim_path_3d[:,[1,0]]
airsim_path_3d[:,2] = -airsim_path_3d[:,2]
airsim_path_3d = airsim_path_3d + center - airsim_start

In [ ]:
fig = plot_path_3d(x=airsim_path_3d[:,0], y=airsim_path_3d[:,1], z=airsim_path_3d[:,2])
fig.update_layout(width=1600, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
np.save('path.npy', airsim_path_3d)

In [ ]:
transform = np.array([[1.0, 0.0, 0.0, -1.0172526572205243e-06],
                      [0.0, 1.0, 0.0, -0.0],
                      [0.0, 0.0, 1.0, -176.6666717529297]])
scale = 0.0025

x_grid = x_grid / scale - transform[0,3]
y_grid = y_grid / scale - transform[1,3]
z_grid = z_grid / scale - transform[2,3]

path_3d_np = path_3d.detach().cpu().numpy() / scale - transform[0:3,3]

In [ ]:
fig = plot_surface(x_grid, y_grid, z_grid, no_axes=False)
fig = plot_path_3d(fig=fig, x=path_3d_np[:,0], y=path_3d_np[:,1], z=path_3d_np[:,2])
fig.show()

In [ ]:
path_metrics(path_3d)

### Dubin's with $\theta$ optimization

In [ ]:
# Compute initial headings
thetas = torch.atan2(path_xy_torch[1:,1] - path_xy_torch[:-1,1], path_xy_torch[1:,0] - path_xy_torch[:-1,0])  
# Duplicate last heading
thetas = torch.cat((thetas, thetas[-1].unsqueeze(0)), dim=0)

path = torch.cat((path_xy_torch, thetas.unsqueeze(1)), dim=1)  # (x, y, theta)
# Fixed variables are initial and final states, free variables are intermediate states
path_start = path[0].clone().detach()
path_end = path[-1].clone().detach()
path_opt = path[1:-1].clone().detach().requires_grad_(True)

In [ ]:
# Dubin's based cost
def cost(path, dt=1.0):
    thetas = path[:,2]  
    omegas = wrap_angle_torch(thetas.diff()) / dt  
    # Path Vs
    path_dxy = torch.diff(path[:,:2], dim=0)
    Vs = torch.norm(path_dxy, dim=1) / dt
    controls_cost = 0.1 * (torch.abs(Vs)).nanmean() + (torch.abs(omegas)).nanmean()
    # Slope cost
    path_zs = 10 * nemo.get_heights(path)
    path_zs -= path_zs.min()
    path_zs = path_zs**2
    slope_cost = 1 * (torch.abs(path_zs.diff(dim=0))).nanmean()
    print(f"controls_cost: {controls_cost}, slope_cost: {slope_cost}")
    return controls_cost + slope_cost

In [ ]:
path_zs = 10 * nemo.get_heights(path)
path_zs -= path_zs.min()
path_zs = path_zs**2
print(path_zs.min(), path_zs.max())
costs = torch.abs(path_zs.diff(dim=0))
print(costs.min(), costs.max())

In [ ]:
# Optimize path
opt = torch.optim.Adam([path_opt], lr=1e-3)

for it in range(500):
    opt.zero_grad()
    path = torch.cat((path_start[None], path_opt, path_end[None]), dim=0)
    c = cost(path)
    c.backward()
    opt.step()
    if it % 50 == 0:
        print(f'it: {it},  Cost: {c.item()}')

print(f'Finished optimization - final cost: {c.item()}')

In [ ]:
path_zs = nemo.get_heights(path[:,:2])
path_3d = torch.cat((path[:,:2], path_zs), dim=1)

In [ ]:
fig = go.Figure()
fig = plot_surface(fig, x_grid, y_grid, z_grid, no_axes=True)
fig = plot_path_3d(fig, x=path_3d[:,0].detach().cpu().numpy(), 
                        y=path_3d[:,1].detach().cpu().numpy(), 
                        z=path_3d[:,2].detach().cpu().numpy())
fig.show()

### Double integrator dynamics

In [ ]:
dt = 0.1
path_vs = torch.diff(path, dim=0) / dt
path_as = torch.diff(path_vs, dim=0) / dt
controls_cost = 2 * (torch.norm(path_as, dim=1)**2).mean()

In [ ]:
def resample_path(path, rate=10):
    """Resample path at higher resolution using double integrator dynamics"""
    path_vs = torch.diff(path, dim=0) / dt
    path_as = torch.diff(path_vs, dim=0) / dt
    path_resampled = [path[0]]
    for i in range(len(path)-1):
        for j in range(rate):
            t = j / rate
            path_resampled.append(path[i] + path_vs[i]*t + 0.5*path_as[i]*t**2)
    print(path[-1])
    path_resampled.append(path[-1])
    return torch.stack(path_resampled)

In [ ]:
resampled_path = resample_path(path, rate=10)
resampled_path

In [ ]:
# Double integrator dynamics
def di_cost(path, dt=0.1):
    path_vs = torch.diff(path, dim=0) / dt
    path_as = torch.diff(path_vs, dim=0) / dt
    path_dxy = torch.diff(path, dim=0)
    Vs = torch.norm(path_dxy, dim=1) / dt
    return torch.mean(Vs**2)

In [ ]:
opt = torch.optim.Adam([path_opt], lr=1e-3)

for it in range(500):
    opt.zero_grad()
    path = torch.cat((path_start[None], path_opt, path_end[None]), dim=0)
    c = dubins_cost(path)
    c.backward()
    opt.step()
    if it % 50 == 0:
        print(f'it: {it},  Cost: {c.item()}')